# Countermeasure to DPA and CPA on AES
In the previous notebook, we designed and implemented a countermeasure against SPA. In this notebook you will learn about masking as a countermeasure to protect AES against CPA's. (You will not need to adjust the ssource code yourself, but we have modified the code for you so you just need to compile and flash the new firmware).

**Goals:**
* Learn to use Chipwhisperer projects
* Use masking as countermeasure against CPA on AES
* Test out the countermeasure

#### Prerequisites
- [x] *0_series* notebooks 
- [x] *1_series* notebooks
- [x] *2_series* notebooks
- [x] *3_A - Countermeasure to SPA on RSA* notebook

## Chipwhisperer projects
To begin we are going to see how we can use CW projects to perform a CPA very easily. All the code used in notebook `2_B - CPA on AES cryptosystem` is already packaged nicely in the CW api (the api provides an even better verion of the attack). In the next code blocks we will program the target with the normal AES firmware and use CW projects to launch an attack an get the secret key:

In [15]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET='AVRCRYPTOLIB' 
SS_VER='SS_VER_1_1'

In [16]:
%run "Setup_Scripts/Setup_Generic.ipynb"

Serial baud rate = 38400
INFO: Found ChipWhisperer😍


In [17]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../chipwhisperer/hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

Building for platform CWLITEXMEGA with CRYPTO_TARGET=AVRCRYPTOLIB
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
rm -f -- simpleserial-aes-CWLITEXMEGA.hex
rm -f -- simpleserial-aes-CWLITEXMEGA.eep
rm -f -- simpleserial-aes-CWLITEXMEGA.cof
rm -f -- simpleserial-aes-CWLITEXMEGA.elf
rm -f -- simpleserial-aes-CWLITEXMEGA.map
rm -f -- simpleserial-aes-CWLITEXMEGA.sym
rm -f -- simpleserial-aes-CWLITEXMEGA.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-aes.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s aes-independant.s aes_enc.s aes_keyschedule.s aes_sbox.s aes128_enc.s
rm -f -- simpleserial-aes.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d aes-independant.d aes_enc.d aes_keyschedule.d aes_sbox.d aes128_enc.d
rm -f -- simpleserial-aes.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i aes-independant.i aes_enc.i aes_keyschedule.i aes_sbox.i aes128_enc.i
.
Welcome to another exciting Chi

.././simpleserial/simpleserial.c:18:0: warning: "CRC" redefined
 #define CRC 0xA6
 ^
In file included from /usr/lib/avr/include/avr/io.h:536:0,
                 from .././hal/hal.h:93,
                 from .././simpleserial/simpleserial.c:5:
/usr/lib/avr/include/avr/iox128d3.h:2240:0: note: this is the location of the previous definition
 #define CRC    (*(CRC_t *) 0x00D0)  /* Cyclic Redundancy Checker */
 ^


In [18]:
cw.program_target(scope, prog, "../../chipwhisperer/hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 3761 bytes


#### Creating the CW project using the api

In [19]:
proj = cw.create_project("AES_normal", overwrite=True)

In [20]:
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

N = 50
for i in tnrange(N, desc='Capturing traces'):
    key, text = ktp.next()
    trace = cw.capture_trace(scope, target, text, key)
    if not trace:
        continue
    
    proj.traces.append(trace)

<ipython-input-20-32882a7d23d7>:10: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(N, desc='Capturing traces'):


#### performing the attack using the api

In [22]:
import chipwhisperer.analyzer as cwa

def CPA_attack(proj):
    leak_model = cwa.leakage_models.sbox_output
    attack = cwa.cpa(proj, leak_model)
    results = attack.run()
    print(results)
    print("The attack reveals the following key guess:\n",end='')
    print(str(bytearray(results.key_guess())),end=' ')

CPA_attack(proj)
print(" ")
print("\nThe real key was:\n"+str(key))

Subkey KGuess Correlation
  00    0x2B    0.89138
  01    0x7E    0.80072
  02    0x15    0.83979
  03    0x16    0.74770
  04    0x28    0.82708
  05    0xAE    0.88814
  06    0xD2    0.85648
  07    0xA6    0.78971
  08    0x70    0.72649
  09    0xF7    0.88605
  10    0x15    0.91154
  11    0x88    0.81249
  12    0x09    0.70952
  13    0xCF    0.75231
  14    0x4F    0.89571
  15    0x3C    0.87645

The attack reveals the following key guess:
CWbytearray(b'2b 7e 15 16 28 ae d2 a6 70 f7 15 88 09 cf 4f 3c')  

The real key was:
CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')


As you can tell, using the CW project is super easy. The only thing to remember is the kind of attack (what kind of leakage model) we wish to use. In our case it is `leak_model = cwa.leakage_models.sbox_output`. We will use the projects to test our new version of the AES to see if it can still crack it.

## Designing a countermeasure
An important part of the countermeasure is to keep te encryption the same. So let's check the encryption of an example input to later compare it to the encrypted text using the countermeasure. Both encrypted texts should be identical for the same plain text input:
(We also need to set the same key)

In [23]:
key_normal_aes = key
plain_text_normal_aes = bytearray([0xf1]*16)
print("The used key in the normal AES was: "+str(key_normal_aes))
print("The plain text that will be encrypted by the normal AES is: "+str(plain_text_normal_aes))
target.simpleserial_write('p', plain_text_normal_aes)

The used key in the normal AES was: CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')
The plain text that will be encrypted by the normal AES is: CWbytearray(b'f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1')


In [24]:
encrypted_text_normal_aes = target.simpleserial_read('r', 16)
print("The resulting encrypted text with normal AES is: "+str(encrypted_text_normal_aes))

The resulting encrypted text with normal AES is: CWbytearray(b'e4 3d 50 f5 d8 6a d7 89 27 84 1a f2 c4 18 80 ed')


Next up is to compile the masked AES firmware, flash it to the target and compare the encrypted text for the same input and key:

In [25]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ./src/aes_masked/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

Building for platform CWLITEXMEGA with CRYPTO_TARGET=AVRCRYPTOLIB
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
rm -f -- simpleserial-aes-CWLITEXMEGA.hex
rm -f -- simpleserial-aes-CWLITEXMEGA.eep
rm -f -- simpleserial-aes-CWLITEXMEGA.cof
rm -f -- simpleserial-aes-CWLITEXMEGA.elf
rm -f -- simpleserial-aes-CWLITEXMEGA.map
rm -f -- simpleserial-aes-CWLITEXMEGA.sym
rm -f -- simpleserial-aes-CWLITEXMEGA.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-aes.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s aes-independant.s aes_enc.s aes_keyschedule.s aes_sbox.s aes128_enc.s
rm -f -- simpleserial-aes.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d aes-independant.d aes_enc.d aes_keyschedule.d aes_sbox.d aes128_enc.d
rm -f -- simpleserial-aes.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i aes-independant.i aes_enc.i aes_keyschedule.i aes_sbox.i aes128_enc.i
.
Welcome to another exciting Chi

.././simpleserial/simpleserial.c:18:0: warning: "CRC" redefined
 #define CRC 0xA6
 ^
In file included from /usr/lib/avr/include/avr/io.h:536:0,
                 from .././hal/hal.h:93,
                 from .././simpleserial/simpleserial.c:5:
/usr/lib/avr/include/avr/iox128d3.h:2240:0: note: this is the location of the previous definition
 #define CRC    (*(CRC_t *) 0x00D0)  /* Cyclic Redundancy Checker */
 ^


In [26]:
cw.program_target(scope, prog, "./src/aes_masked/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 3795 bytes


In [27]:
ktp = cw.ktp.Basic()
key, text = ktp.next()

target.set_key(key)
key_masked_aes = key
plain_text_masked_aes = bytearray([0xf1]*16)
print("The used key in the masked AES was: "+str(key_masked_aes))
print("The plain text that will be encrypted by the masked AES is: "+str(plain_text_masked_aes))
target.simpleserial_write('p', plain_text_masked_aes)

The used key in the masked AES was: CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')
The plain text that will be encrypted by the masked AES is: CWbytearray(b'f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1 f1')


In [28]:
encrypted_text_masked_aes = target.simpleserial_read('r', 16)
print("The resulting encrypted text with normal AES is: "+str(encrypted_text_masked_aes))

The resulting encrypted text with normal AES is: CWbytearray(b'e4 3d 50 f5 d8 6a d7 89 27 84 1a f2 c4 18 80 ed')


So the encryption still works exactly the same. The next step is to go over the changes in the masked AES and finally test if we can crack it. 
The reason we were able to crack the normal AES is that there was a correlation between the powerconsumtion and the data set on the databusses. We could predict what the value of a databit would be if we guessed the correct key. By comparing this to the measured power traces we could quantify the probability of our guess being correct. In order to make it impossible for us to know the value of the databus internally we will use a strategy called masking. Masking means that we perform an operation between the plain text and a randomly generated number. The operation must be choosen so that at the final step we can use that same random number to peel back the mask and reveal the correct encryption. A good operation for this is the `XOR` operation because to reverse an xor with a number you just have to repeat the xor operation with the same number. 
```
byte = 0xF1 = 0b1111 0001
mask = 0x55 = 0b0101 0101

0xF1^0x55 = 0xA4 = 0b1010 0100
0xA4^0x55 = 0xF1 = 0b1111 0001
```
Furthermore when performing linear operations on the masked data the same random number can be used to unmask the data. For example if we `xor` the masked data with a keybyte we can easily get the unmasked xor between the key by unmasking with the same mask:
```
normal: 0xF1 ^ 0x2B        = 0xDA = 0b1101 1010
masked: (0xF1^0x55) ^ 0x2B = 0x8F = 0b0100 1111
unmasking: 0x8F ^ 0x55     = 0xDA
```
This means we can perform the AES operations while not knowing the inner bit values because they depend on the random mask. There is one problem: the SBOX operation does not keep the mask. We can circumvent this problem by unmasking the data before sending it into the SBOX and remasking its output. Just executing those steps sequentially in code is not an option however. The buslines would hold the same data as in the normal AES for a split second. The means the mask would be useless, but we can instead make a different SBOX based on the random mask that already contains this unmasking and remasking of the data. That way the databusses never get the value as in the normal AES because the new SBOX is created ahead of time right after the random mask is made. Let's check out what this would look like:
```
//Sequential (unsafe):
- sbox_input = masked_input ^ mask       //unmask the byte before using the sbox
- sbox_out   = sbox(sbox_input)          //performing sbox operation
- remasked_sbox_output = sbox_out ^ mask //remasking the output

//Masking new SBOX based on mask
for(i=0; i<length(sbox); ++i){
        sbox2[i^mask] = aes_sbox[i]^mask;
	}   

//then just use the following
remasked_sbox_output = sbox2[masked^input]

//EXAMPLE:
sbox[0xDA] = 0x57  //normal aes

sbox2[0xDA^0x55] = sbox2[0x8F] = 0x02 (= sbox[0xDA]^0x55) //masked
0x02 ^ 0x55 = 0x57 //after unmasking is same as normal aes
```
To see how these changes to AES are implemented in the Masked AES you can open the 2 changed source files here: 

[simpleserial-aes.c](http://localhost:8888/edit/IIW/UHasselt%20Chipwhisperer%20notebooks/src/aes_masked/simpleserial-aes/simpleserial-aes.c)

[aes_enc.c](http://localhost:8888/edit/IIW/UHasselt%20Chipwhisperer%20notebooks/src/aes_masked/crypto/avrcryptolib/aes/aes_enc.c)

The original source files of the normal AES you can open here:

[simpleserial-aes.c](http://localhost:8888/edit/chipwhisperer/hardware/victims/firmware/simpleserial-aes/simpleserial-aes.c)

[aes_enc.c](http://localhost:8888/edit/chipwhisperer/hardware/victims/firmware/crypto/avrcryptolib/aes/aes_enc.c)

Some other instructions used in the aes firmware are non-linear which means we again need to unmask and remask data. By performing the unmasking and remasking in the same instruction the databusses again don't take the values as in the normal aes and thus protect the leakage.

## Testing the masked AES
The final step is to try and crack our masked AES. We will use a CW project again, but this time it's up to you to write the correct code:

In [29]:
#1.Start your code here
###START SOLUTION###
proj = cw.create_project("AES_masked", overwrite=True)
###END SOLUTION###

In [30]:
#2.Start your code here
###START SOLUTION###
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

N = 50
for i in tnrange(N, desc='Capturing traces'):
    key, text = ktp.next()
    trace = cw.capture_trace(scope, target, text, key)
    if not trace:
        continue
    
    proj.traces.append(trace)
###END SOLUTION###

<ipython-input-30-f18dab3a3a14>:12: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(N, desc='Capturing traces'):


In [31]:
#3.Start your code here
###START SOLUTION###
CPA_attack(proj)
print(" ")
print("\nThe real key was:\n"+str(key))
###END SOLUTION###

Subkey KGuess Correlation
  00    0x2E    0.66748
  01    0x60    0.57909
  02    0x87    0.64625
  03    0x48    0.63583
  04    0x6F    0.62046
  05    0x3C    0.67086
  06    0x6D    0.66044
  07    0x33    0.64350
  08    0x79    0.64775
  09    0x34    0.64879
  10    0x5E    0.66954
  11    0xFF    0.62453
  12    0x1E    0.64359
  13    0xFA    0.61481
  14    0xB4    0.60430
  15    0x6D    0.64917

The attack reveals the following key guess:
CWbytearray(b'2e 60 87 48 6f 3c 6d 33 79 34 5e ff 1e fa b4 6d')  

The real key was:
CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')


## Conclusion
By using a mask the internal data does not correlate with the predicted data. That way a CPA attack is prevented from finding the secret key.

## Clean up

In [ ]:
scope.dis()
target.dis()

## THE END
Congratulations on finishing this tutorial series and feel free to experiment further with the Chipwhisperer.